In [ ]:
# 1. Clean old versions
!pip uninstall -y unsloth transformers

# 2. Install known-compatible transformer version (4.36.2) + Unsloth
!pip install transformers==4.36.2
!pip install unsloth==2024.4.1

# 3. Install required libs
!pip install bitsandbytes accelerate peft trl datasets


Found existing installation: transformers 4.36.2
Uninstalling transformers-4.36.2:
  Successfully uninstalled transformers-4.36.2
  Using cached transformers-4.36.2-py3-none-any.whl.metadata (126 kB)
Using cached transformers-4.36.2-py3-none-any.whl (8.2 MB)
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
sentence-transformers 4.1.0 requires transformers<5.0.0,>=4.41.0, but you have transformers 4.36.2 which is incompatible.
ERROR: Could not find a version that satisfies the requirement unsloth==2024.4.1 (from versions: 2024.8, 2024.9, 2024.9.post1, 2024.9.post2, 2024.9.post3, 2024.9.post4, 2024.10.0, 2024.10.1, 2024.10.2, 2024.10.4, 2024.10.5, 2024.10.6, 2024.10.7, 2024.11.2, 2024.11.4, 2024.11.5, 2024.11.6, 2024.11.7, 2024.11.8, 2024.11.9, 2024.11.10, 2024.11.11, 2024.12.1, 2024.12.2, 2024.12.3, 2024.12.4, 2024.12.5, 2024.12.6, 2024.12.7, 2024.12.8, 2024.12.9

In [ ]:
!pip uninstall -y unsloth
!rm -rf /content/unsloth  # if running on Colab, clear any cached files
!pip install -U git+https://github.com/unslothai/unsloth.git@main#egg=unsloth --no-deps --force-reinstall


Found existing installation: unsloth 2025.6.9
Uninstalling unsloth-2025.6.9:
  Successfully uninstalled unsloth-2025.6.9
  Cloning https://github.com/unslothai/unsloth.git (to revision main) to /tmp/pip-install-9ytkf5dq/unsloth_5fb2e3f7618749d19c8232ed03971686
  Running command git clone --filter=blob:none --quiet https://github.com/unslothai/unsloth.git /tmp/pip-install-9ytkf5dq/unsloth_5fb2e3f7618749d19c8232ed03971686
  Resolved https://github.com/unslothai/unsloth.git to commit 9aa13e4e64ccce0d89c726685fdf0b99ecfaa0a8
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for unsloth: filename=unsloth-2025.6.9-py3-none-any.whl size=294271 sha256=4130be1d99d8630d01693b63103be9e5f75c5c70a4670114036b3b9fc6ef0329
  Stored in directory: /tmp/pip-ephem-wheel-cache-aocr6q6l/wheels/a9/37/63/79f42d6e25c25362a50966663e74c2e07b3d81235794a5cf88
Successfully built unsloth


In [ ]:

from unsloth import FastLanguageModel
print("Unsloth imported successfully ✅")


ModuleNotFoundError: No module named 'unsloth_zoo.utils'

In [ ]:
# MUST be run before loading anything
import os
os.environ["DISABLE_TRITON"] = "1"
os.environ["FLASH_ATTENTION_FORCE_DISABLED"] = "1"
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "max_split_size_mb:128"


In [ ]:

from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd

# Load your .tsv file
data_path = "/content/drive/MyDrive/MentalQA/Train_Dev.tsv"
df = pd.read_csv(data_path, sep="\t")

# Preview first few rows
df.head()


,question,answer,final_QT,final_AS
0,هل يعتبر الخوف من عدم الإنجاب مستقبلاً حالة عا...,سد تى نعم ان هناك العديد من القلق النفسى المرت...,"['A', 'D']","['1', '2']"
1,من سنه تقريبا و انا أذي نفسي ب اكثر من طريقة و...,يجب العرض علي طبيب اختصاصي امراض نفسية سوف يقو...,"['B', 'E']",['2']
2,السلام عليكم مشكلتي تقتصر على تكرار كلمة معينة...,الوسواس القهري هو من الامراض الشاءعة ويكون في ...,"['A', 'E']","['1', '2']"
3,اكتئاب وفوبيا من المجتمع وانعزال وانطوائية وتع...,العلاج النفسي المعرفي السلوكي يعطي نتائج جيدة ...,"['B', 'D']",['1']
4,هل الإحساس بقرب الاجل و الخوف من الموت و الاحل...,نعم بالاضافه للكثير من الاعراض الاخرى الطبيب ا...,"['A', 'B', 'D']","['1', '2']"


In [ ]:
# Build instruction-format dataset
formatted_dataset = []

for _, row in df.iterrows():
    formatted_dataset.append({
        "instruction": "أجب عن السؤال التالي",
        "input": row["question"],
        "output": row["answer"]
    })

# Save to JSONL
import json

output_path = "/content/drive/MyDrive/MentalQA/mentalqa_instruct.jsonl"
with open(output_path, "w", encoding="utf-8") as f:
    for item in formatted_dataset:
        f.write(json.dumps(item, ensure_ascii=False) + "\n")

print(f"Saved {len(formatted_dataset)} examples to {output_path}")


Saved 350 examples to /content/drive/MyDrive/MentalQA/mentalqa_instruct.jsonl


In [ ]:
from unsloth import FastLanguageModel
import torch

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Qwen3-1.7B-unsloth-bnb-4bit",
    max_seq_length = 2048,
    dtype = torch.float16,
    load_in_4bit = True,
)


ModuleNotFoundError: No module named 'unsloth'

In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 64,
    lora_alpha = 16,
    lora_dropout = 0.0,  # 0 = fastest with full Unsloth patching
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj"]
)

# Optional: check how many LoRA params are trainable
trainable = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"Trainable LoRA parameters: {trainable:,}")


In [ ]:
from datasets import load_dataset

# Load your JSONL instruction dataset
dataset = load_dataset("json", data_files="/content/drive/MyDrive/MentalQA/mentalqa_instruct.jsonl", split="train")

# Split into train and small eval set (5%)
train_dataset = dataset.train_test_split(test_size=0.05, seed=42)["train"]
eval_dataset  = dataset.train_test_split(test_size=0.05, seed=42)["test"]


In [ ]:
# Define the Qwen-style prompt format
def formatting_func(example):
    return f"""### Instruction:
{example['instruction']}
### Input:
{example['input']}
### Response:
{example['output']}"""

# Tokenize the dataset
def tokenize(example):
    return tokenizer(
        formatting_func(example),
        truncation=True,
        padding="max_length",
        max_length=2048,
    )

train_dataset = train_dataset.map(tokenize)
eval_dataset = eval_dataset.map(tokenize)


In [ ]:
from transformers import TrainingArguments
from trl import SFTTrainer

training_args = TrainingArguments(
    output_dir = "qwen3-mentalqa-lora",
    num_train_epochs = 3,
    per_device_train_batch_size = 2,
    gradient_accumulation_steps = 4,
    learning_rate = 2e-4,
    fp16 = True,
    logging_steps = 20,
    save_total_limit = 1
)

trainer = SFTTrainer(
    model = model,
    train_dataset = train_dataset,
    eval_dataset = eval_dataset,
    dataset_text_field = None,
    tokenizer = tokenizer,
    args = training_args,
    max_seq_length = 2048,
    formatting_func = formatting_func,
)

trainer.train()
